# Earnings Call Grading Notebook

This notebook will take all the earnings calls that are saved in Snowflake daily, iterate thru each call and grade each call.

In [ ]:
# Import necessary packages
import streamlit as st
import pandas as pd
import requests
import json
from io import StringIO
import re
import math
from datetime import date, datetime, timedelta

# Import Snowpark session
from snowflake.snowpark.context import get_active_session

# Initialize Snowpark session
session = get_active_session()

# Configuration parameters
DATE_STR = str(date.today())
DAYS_TO_ITERATE = 3
SPLIT_TOKENS = 125000  # Max number of tokens to send to summary LLM Call
AI_MODEL = 'llama3.1-405b'

# Print the configuration parameters for verification
print(f"Configuration - Date: {DATE_STR}, Days to Iterate: {DAYS_TO_ITERATE}, Split Tokens: {SPLIT_TOKENS}, AI Model: {AI_MODEL}")


## Call Check
Check which alls are not graded and grade only the calls which are not yet graded.

In [ ]:
-- with T_MINUS as(
--     select STOCK_SYMBOL, FISCAL_QUARTER::number as FQ, FISCAL_YEAR::Number AS FY
--     from EARNINGS.PUBLIC.TRANSCRIPTS
--     MINUS
--     select STOCK_SYMBOL, FISCAL_QUARTER::Number, FISCAL_YEAR::Number
--     from EARNINGS.ANALYSIS.EARNINGS_SUMMARIES
-- )
-- select TR.*
-- from EARNINGS.PUBLIC.TRANSCRIPTS TR inner join T_MINUS T
-- on T.STOCK_SYMBOL = TR.STOCK_SYMBOL
-- AND T.FQ = TR.FISCAL_QUARTER 
-- AND T.FY = TR.FISCAL_YEAR

In [ ]:

getSQL = f'''
    with T_MINUS as(
        select STOCK_SYMBOL, FISCAL_QUARTER::number as FQ, FISCAL_YEAR::Number AS FY
        from EARNINGS.PUBLIC.TRANSCRIPTS
        MINUS
        select STOCK_SYMBOL, FISCAL_QUARTER::Number, FISCAL_YEAR::Number
        from EARNINGS.ANALYSIS.EARNINGS_SUMMARIES
    )
    select TR.*
    from EARNINGS.PUBLIC.TRANSCRIPTS TR inner join T_MINUS T
    on T.STOCK_SYMBOL = TR.STOCK_SYMBOL
    AND T.FQ = TR.FISCAL_QUARTER 
    AND T.FY = TR.FISCAL_YEAR
    ORDER BY QUERY_DATE DESC
'''

TranscriptsToProcess = session.sql(getSQL)
print(TranscriptsToProcess.toPandas())

## Extract JSON Helper Function

This function designed to sanitize and extract JSON data from a given input string, typically containing unstructured or semi-structured data that may include special characters and formatting issues.

This function and others like it are critical for Gen-AI and LLM applications because their output may not be as expected.

For example the LLM may believe it is speaking to a human and say something like:

```
Below is your JSON
    {JSON here:[]}
Let me know if you need anything else
```

The above may have valid JSON within it but the english header/footer are break the code. 

In [ ]:
def extractJSON(result):
    # Replace problematic characters
    sanitized_result = re.sub(r'[\n\r\u0085\u2028\u2029]+', ' ', str(result))
    sanitized_result = re.sub(r'" "sections"', '", "sections"', sanitized_result)

    # Extract JSON from the sanitized string
    json_match = re.search(r'{.*}', sanitized_result, re.DOTALL)
    
    if json_match:
        json_str = json_match.group()
        try:
            final_summary = json.loads(json_str)
            return final_summary
        except json.JSONDecodeError as e:
            print("Sanitized Result:", sanitized_result)
            print(f"JSONDecodeError: {e}")
    else:
        print("No valid JSON found in the result.")

## Splitter Function
Depending on your the model the call log may need to be split into smaller chunks and send to the LLM function via many iterations. Different functions have different [context windows](https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions#model-restrictions). 

The **SPLIT_TOKENS = 30000** and **AI_MODEL = 'mistral-large'** are critical for the notebook and the function below.

In [ ]:
def split_into_chunks(text):
    # Query to calculate the number of tokens required to split the text
    query = f"SELECT CEIL(SNOWFLAKE.CORTEX.COUNT_TOKENS( '{AI_MODEL}', '{text}' )/{SPLIT_TOKENS}) as TOKEN_LEN;"
    token_split_len = session.sql(query).to_pandas().iloc[0]['TOKEN_LEN']
    
    print('splitting - ', end=" ")
    
    # Tokenize the text using regular expressions to capture words and punctuation
    tokens = re.findall(r'\S+|\n', text)
    
    chunks = []
    current_chunk = []
    current_length = 0
    
    # Calculate the maximum number of tokens per chunk
    max_split_tokens = int(math.ceil(len(text) / token_split_len))
    
    print(f'({token_split_len}, {max_split_tokens}) -', end=" ")
    
    for token in tokens:
        token_length = len(token) + 1  # Adding 1 for the space or punctuation
        
        if current_length + token_length >= max_split_tokens:
            # Join current_chunk into a string and append to chunks
            chunks.append(' '.join(current_chunk))
            # Start a new chunk
            current_chunk = [token]
            current_length = token_length
        else:
            # Add the token to the current chunk
            current_chunk.append(token)
            current_length += token_length
    
    # Append the last chunk if it has any tokens
    if current_chunk:
        chunks.append(' '.join(current_chunk))
    
    return chunks

## Get Transcript
Go to the transcript table and get the transcript

In [ ]:
def getTranscript(ticker, querydate):
    print(f'{ticker} | Transcript - ', end=" ")

    SQLTxt = f"""
        SELECT 
            STOCK_SYMBOL, 
            FISCAL_QUARTER, 
            FISCAL_YEAR, 
            TRANSCRIPT, 
            RELEASE_DATE
        FROM 
            EARNINGS.public.transcripts
        WHERE 
            stock_symbol = '{ticker}'
            AND query_date = '{querydate}'
    """

    try:
        result = session.sql(SQLTxt).collect()
        if result:
            transcript = str(result[0]['TRANSCRIPT'])
            fiscal_quarter = result[0]['FISCAL_QUARTER']
            fiscal_year = result[0]['FISCAL_YEAR']
            release_date = str(result[0]['RELEASE_DATE'])
            return transcript, fiscal_quarter, fiscal_year, release_date, ''
            
        else:
            return '', '', '', '', ''
    except Exception as e:
        print(f"Error fetching transcript: {e}")
        return '', '', '', '', ''


# Summarize using Cortex Complete 
In this function we actually grade and pull out any relevent information from the financial call.

In the prompt below we provide **instructions** for the LLM on what actions to perform.

we also provide the **Segment** which is the call it self or portions of the call.

The LLM is then asked to **output** any specific information we are asking for such as performance, initiatives, challenges and any annoucements from the call. 

In [ ]:
def splitTransacript(escaped_transcript):
    chunks = split_into_chunks(escaped_transcript )
    
    # Summarize each chunk
    results = []
    for chunk in chunks:
        MLTXT = f"""
            SELECT SNOWFLAKE.CORTEX.COMPLETE(
                '{AI_MODEL}',
                ARRAY_CONSTRUCT(
                    OBJECT_CONSTRUCT(
                        'role', 'user',
                        'content', '### INSTRUCTIONS
                            You will be provided with a segment of a financial earnings call transcript. 
                            Your task is to summarize this segment accurately and concisely, focusing on key points such as financial performance, strategic initiatives, Challenges, significant announcements, and any other relevant information.
                            Ensure the summary is clear and easy to understand for a broad audience, including investors, analysts, and business professionals.

                            Be descriptive in your analysis, do not worry about providing short summary. 
                            
                            <Segment>{chunk}</Segment>

                            ### OUTPUT
                            Provide a well-structured summary in the following format:
                            0. Please make sure to include the company name.
                            1. Financial Performance: ...
                            2. Strategic Initiatives: ...
                            3. Challenges: ...
                            4. Significant Announcements: ...
                            5. Other Relevant Information: ...

                            Only include the summary and nothing else.
                            
                            ### EXCLUDE
                            DO NOT any new lines or any characters that will invalidate JSON
                            Do not mention or write COMPANY ABC or COMPANY XYZ use the company name.
                        '
                    )
                ),
                OBJECT_CONSTRUCT('temperature', 0.7)
            ) AS result
        """
        
        result = session.sql(MLTXT).collect()
        results.append(result[0]['RESULT'])
    
    return results


# Summarize and Grade 

Below we ask the LLM to take all the extracted information above and summarize it into a relevent cohesive JSON output.

We ask the LLM to be a world-class financial report and provide us with the **title of the article**, an introduction, detailed analysis of the call and then any positives or negatives about the call. 

This condensed portion of the call will be used to display in a streamlit application.

In [ ]:
def summarizeTransacript(results):
    print('summarizing - ', end =" ")
    # Convert the results to JSON
    results_json = [json.loads(result) for result in results]

    # Extract just the messages from the results
    extracted_messages = ' '.join([res['choices'][0]['messages'] for res in results_json])
    
    ex_msg_escaped = extracted_messages.replace('\\', '\\\\').replace('\'', '\\\'')

    content_str = f"""
'### INSTRUCTIONS
You are a world-class financial reporter. 
You will be provided with several summaries from a financial earnings call transcript. 
Your task is to combine and refine these summaries into a single, coherent article structured in JSON format. 

The JSON should include a title, section titles, and section bodies. 
The article should start with an engaging introduction, followed by detailed analysis, the positives, the negatives, and conclude with a summary of the key points.

Each section should be comprehensive, detailed, and at least 5-7 sentences in length, providing in-depth insights and analysis.

{ex_msg_escaped}

### OUTPUT
Only provide the output in the following JSON format without any new lines or extra characters 
    and nothing else, no extra comments JUST THE JSON:
{{
  "title": "Title of the Article",
  "sections": [
    {{
      "section_title": "Introduction",
      "section_body": "..."
    }},
    {{
      "section_title": "Detailed Analysis",
      "section_body": "..."
    }},
    {{
      "section_title": "Positives",
      "section_body": "..."
    }},
    {{
      "section_title": "Negatives",
      "section_body": "..."
    }},
    {{
      "section_title": "Conclusion",
      "section_body": "..."
    }}
  ]
}} 

### EXCLUDE
DO NOT include any new lines or any characters that will invalidate JSON.
'
"""
    
    query = f"SELECT SNOWFLAKE.CORTEX.COUNT_TOKENS( '{AI_MODEL}', {content_str} ) as TOKEN_LEN;"
    token_len = session.sql(query).to_pandas().iloc[0]['TOKEN_LEN']

    if(token_len >= SPLIT_TOKENS-1):
        print('----------------------------------------------------------------')
        print(f'Token Length Summary Overload: {token_len}')
        return "{}"
        
    else:
        # Combine and refine results
        final_summary_query = f"""
            SELECT SNOWFLAKE.CORTEX.COMPLETE(
                '{AI_MODEL}',
                ARRAY_CONSTRUCT(
                    OBJECT_CONSTRUCT(
                        'role', 'user',
                        'content', {content_str}
                    )
                ),
                OBJECT_CONSTRUCT('temperature', 0.7)
            ) AS RESULT
        """
        
        # Execute the final summary query
        final_summary = session.sql(final_summary_query).collect()[0]['RESULT']
        final_summary = re.sub(r'[\n\r\u0085\u2028\u2029]+', ' ', final_summary)
        return final_summary

## Get Additional Company Information

In [ ]:
def getCompanyProfile(ticker):
    print('Profile - ', end=" ")
    
    def fetch_data(url):
        query = f"SELECT earnings.public.generic_url('{url}') AS DATA"
        q_res = session.sql(query).to_pandas().iloc[0].iloc[0]

        def remove_single_quotes_within_double_quotes(match):
            return match.group(0).replace("'", "")

        pattern = r'"[^"]*"'
        cleaned_text = re.sub(pattern, remove_single_quotes_within_double_quotes, q_res)

        # Check if q_res is an empty list directly
        if str(cleaned_text) == '[]':
            return None
        return cleaned_text.replace('"', "'")

    def clean_profile_description(profile):
        profile_split = re.split(r'"', profile)

        if len(profile_split) > 1:
            # Replace single quotes and remove unwanted characters
            new_desc = profile_split[1].replace("'", "").replace('\\x', '')
            return profile_split[0] + "'" + new_desc + "'" + profile_split[2]
        return profile

    def parse_profile(profile):
        try:
            profile_cleaned = clean_profile_description(profile)
            profile_json = profile_cleaned.replace("'", '"').replace('None', 'null').replace('True', '"true"').replace('False', '"false"')
            return json.loads(profile_json)[0]
        except json.JSONDecodeError as e:
            print(f'{ticker} - {e}')
            return '{}'

    profile_url = f"https://url/{ticker}"
    profile = fetch_data(profile_url)

    # Return empty JSON object if profile is None, otherwise parse the profile
    return '{}' if profile is None else parse_profile(profile)

# ticker = "TSLA"
# print(getCompanyProfile(ticker))

# Grade Summary
Based on the summary we have above, we will ask the LLM to be a world class financial analyst and grade the call for us.

The grade will be from ***A+*** to ***F*** based on the investibility of the asset based on the grade of the call.

In [ ]:
def getGrade(summary, ticker):
    print("Grading - ", end =" ")
    
    summary = json.loads(summary)['choices'][0]['messages']
    summary_json = extractJSON(summary)

    json_string = json.dumps(summary_json)
    escaped_json_string = json_string.replace('\\', '\\\\').replace('"', '\\"').replace("'", "\\'")



    content_str = f""" 
'### INSTRUCTIONS
You are a world class financial analyst,
You will be provided with a summary of a financial call.
your task is to grade the call from A+ to F on whether to invest in the asset or not,
You must be harsh in your criteria for grading as peoples real funds are at stake. 

The summary is: {escaped_json_string}

### OUTPUT
Only provide the output in the following JSON format without any new lines or extra characters 
    and nothing else, no extra comments JUST THE JSON:
{{
  "grade": "your grade",
  "reason": "your reasoning for the grade"
}}

### EXCLUDE
DO NOT include anything that will invalidate proper JSON
Do not mention COMPANY ABC or COMPANY XYZ. Use the company ticker or company name if you know it. The company ticker is {ticker}
'
"""

    
    query = f"SELECT SNOWFLAKE.CORTEX.COUNT_TOKENS( '{AI_MODEL}', {content_str} ) as TOKEN_LEN;"
    token_len = session.sql(query).to_pandas().iloc[0]['TOKEN_LEN']

    if(token_len >= SPLIT_TOKENS-1):
        print('----------------------------------------------------------------')
        print(f'Token Length Summary Overload: {token_len}')
        return "{}"
        
    else:
        
        grade_query = f"""
            SELECT SNOWFLAKE.CORTEX.COMPLETE(
                '{AI_MODEL}',
                ARRAY_CONSTRUCT(
                    OBJECT_CONSTRUCT(
                        'role', 'user',
                        'content', {content_str}
                    )
                ),
                OBJECT_CONSTRUCT('temperature', 0.7)
            ) AS RESULT
        """
        
        # Execute the final summary query
        finalGrade = json.loads(session.sql(grade_query).collect()[0]['RESULT'])['choices'][0]['messages']
    
        finalGrade = re.sub(r'[\n\r\u0085\u2028\u2029]+', ' ', finalGrade)
        grade_json = extractJSON(finalGrade)
        return grade_json

# Save the Report & the Grade to Snowflake
Give all the information to this function to save the data to ***EARNINGS_SUMMARIES*** table in Snowflake.


In [ ]:
def saveReport (final_summary, ticker, quarter, year, release, grade, earnings_actual, profile, date):  
    print('saving - ', end =" ")
    summary = re.sub(r'[\n\r\u0085\u2028\u2029]+', ' ', final_summary)
    summary = json.loads(final_summary)['choices'][0]['messages']
    summary_json = extractJSON(summary)

    # Extract the title and summary JSON
    title = summary_json["title"]
    sections = summary_json['sections']
    
    # Create a DataFrame for insertion
    data = {
        "STOCK_SYMBOL": [ticker],
        "TITLE": [title],
        "FISCAL_QUARTER": [quarter ],
        "FISCAL_YEAR": [ year ],
        "RELEASE_DATE": [ release ],
        "QUERY_DATE": [ date ],
        "GRADE": [ json.dumps(grade) ],
        "EARNINGS": [ json.dumps(earnings_actual) ],
        "PROFILE": [ json.dumps(profile) ],
        "SUMMARY_JSON": [json.dumps(sections)]
    }
    df = pd.DataFrame(data)
    
    session.use_database("EARNINGS")
    session.use_schema("ANALYSIS")
    # Write the DataFrame to the Snowflake table using the fully qualified table name
    return session.write_pandas(df, "EARNINGS_SUMMARIES", auto_create_table=False)



# Run Everything
Use all the functions above to run everything

In [ ]:
# transcripts = GetTranscriptsToProcess.to_pandas()
transcripts = TranscriptsToProcess.to_pandas()

for index, row in transcripts.iterrows():
    try:
        ticker = row['STOCK_SYMBOL']
        transcript = row['TRANSCRIPT']
        q = row['FISCAL_QUARTER']
        y = row['FISCAL_YEAR']
        r = row['RELEASE_DATE']
        qd = row['QUERY_DATE']
        ea = ''
    
        print(f'{index} - {ticker} - {y} - {q}', end =" ")
        summary = splitTransacript(transcript)
        report = summarizeTransacript(summary)
        profile = getCompanyProfile(ticker)
        grade = getGrade(report, ticker)
        print(grade)
        
        res = saveReport(report, ticker, q, y, r, grade, ea, profile, qd)
        
        print(f' {res}')
    
    except TypeError as e:
        print(f"TypeError for ticker {ticker}: {e}") 
    except Exception as e:
        print(f"Exception for ticker {ticker}: {e}")